Vehicle Fleet Transfer Policies

In [218]:
import random
import numpy as np
from scipy.stats import lognorm

'''
Location object created for each city.
0 ~ A
1 ~ B
2 ~ C
'''
class Location:
    def __init__(self, vehicle_count, index, lowerLimit=None, upperLimit=None):
        self.index = index # 0, 1, or 2
        self.name = ['A', 'B', 'C'][index]
        self.lower = lowerLimit
        self.upper = upperLimit

        self.count = vehicle_count
        self.incoming = 0   # Vehicles on their way to this city

    def __str__(self):
        return f"{self.name} ({self.count})"

    def processNewCars(self):
        self.count += self.incoming
        self.incoming = 0

'''
Simulation for the default model with no transfers (u = x)
'''
class BasePolicy:
    def __init__(self, state):
        self.n = sum(state) # fleet size
        self.A = Location(state[0], 0)
        self.B = Location(state[1], 1)
        self.C = Location(state[2], 2)
        self.initCounts = state
        self.locs = [self.A, self.B, self.C]

        self.demandFunction = lambda: round(np.random.lognormal(2, 0.5)) # Same demand function for all 9 demands
        # self.demandFunction = lambda: round(np.random.lognormal(3, 0.25)) # Same demand function for all 9 demands
        self.lowerRate = 100 # Price to rent a car locally
        self.higherRate = 150 # Price to rent a car and return it in a different city
        self.transferCost = 50  # Cost to transfer each vehicle overnight
        self.policyName = "Base      "

        self.plan = [0, 0, 0] # Planned increase at each city
        self.W = None   # Matrix of demands. W[i][j] is the demand from city i to city j
        self.profit = 0 # Total profit the company has made so far
        self.history = []


    def state(self):
        return (self.A.count, self.B.count, self.C.count)

    def showState(self):
        print(f"{self.policyName}   {self.A}, {self.B}, {self.C}, Profit: ${round(self.profit/1000)} K \n")

    def stageSummary(self, state, action):
          summary = {'x': state, 'u': action}
          self.history.append(summary)

    def showHistory(self):
      for summary in self.history:
        print(summary)

    def generateDemands(self):
        self.W = [[self.demandFunction() for _ in range(3)] for _ in range(3)]

    '''
    Do our best to satisfy demands at each city, prioritizing the higher-rate rentals. Each city has a different
    neighboring city to prioritize.
    '''
    def satisfyDemands(self):
        # print(self.W, "\n...satisfying customer demand...")
        for loc in self.locs:
            self.satisfyDemandFromCity(loc)

        for loc in self.locs:
            loc.processNewCars()

    def satisfyDemandFromCity(self, start):
        demands = self.W[start.index]
        for i in range(1, 4):
            end = self.locs[(start.index + i) % 3] # Prioritize demand to the other two cities
            self.satisfyDemandFromCityToCity(start, end, demands[end.index])

    def satisfyDemandFromCityToCity(self, start, end, count):
        count = min(start.count, count)
        self.profit += count * (self.lowerRate if (start == end) else self.higherRate)
        start.count -= count
        end.incoming += count
        #print(f"moving {count} cars from {start} to {end}")

    def cost(self, plan):
        return sum(map(abs, plan))*self.transferCost/2

    def transferCars(self):
        # print("...transfering cars...")
        for i in range(3):
            self.locs[i].count += self.plan[i]
        self.profit -= self.cost(self.plan)

    # Default Policy: do nothing
    def generatePlan(self):
        self.plan = [0, 0, 0]

    def step(self):
      stageState = self.state()
      self.generatePlan()
      self.transferCars()
      stageAction = self.state()
      # self.showState()
      self.generateDemands()
      self.satisfyDemands()
      # self.showState()
      self.stageSummary(stageState, stageAction)

'''
Simulation with a policy that transfers cars to recreate the starting counts between cities.
'''
class ResetPolicy(BasePolicy):
    def __init__(self, state):
        super().__init__(state)
        self.policyName = "Reset     "

    def generatePlan(self):
        self.plan = [self.initCounts[loc.index] - loc.count for loc in self.locs]

'''
Simulation with a policy that transfers only half of the cars needed to recreate the starting counts between cities.
'''
class HalfResetPolicy(BasePolicy):
    def __init__(self, state):
        super().__init__(state)
        self.policyName = "Half Reset"

    def generatePlan(self):
        odd_adjustment = random.choice([-1, 1])
        for loc in self.locs:
            diff = self.initCounts[loc.index] - loc.count
            if (diff % 2 == 1):
                diff += odd_adjustment # adjust odd differences to make them even
                odd_adjustment *= -1 # the next adjustment should be in the opposite direction so that the plan sums to 0
            self.plan[loc.index] = diff//2

'''
Simulation where each city's vehicle count is brought within some range [loc.lower, loc.upper]
'''

class TwoLimitPolicy(BasePolicy):
    def __init__(self, state, lower=17, upper=24):
        super().__init__(state)
        self.policyName = "Two Limit "
        for loc in self.locs:
            loc.lower = lower
            loc.upper = upper

    def generatePlan(self):
        nextCount = [min(max(loc.lower, loc.count), loc.upper) for loc in self.locs]    # Bring all cities within limits
        nextCount = self.correctDistribution(nextCount) # Adjust to account for all cars
        self.plan = [nextCount[loc.index] - loc.count for loc in self.locs] # How many cars are gained/lost at each city

    # Make sure the total number of cars is still n
    def correctDistribution(self, nextCount):
        if sum(nextCount) == self.n:
            return nextCount

        small = nextCount.index(min(nextCount))
        temp = [nextCount[i] if i!= small else -1 for i in range(3)]
        large = temp.index(max(temp))
        medium = 3 - (small + large)

        # We have extra cars that need to go somewhere - should be transfered to the smallest cities
        if self.n > sum(nextCount):
            excess = self.n - sum(nextCount)
            gap = medium - small
            if gap >= excess: # medium city is large enough that all extra cars go to smallest city
                nextCount[small] += excess
            else: # distribute cars so that medium and small become the same size
                total = self.n - nextCount[large]
                nextCount[medium] = total//2
                nextCount[small] = self.n - (nextCount[large] + nextCount[medium])

        # We need more cars sent to the smallest cities
        else:
            deficit = sum(nextCount) - self.n
            gap = large - medium
            if gap >= deficit: # medium city is small enough that all cars should be taken from the largest city only
                nextCount[large] -= deficit
            else: # take cars from both large and medium, so that they become the same size
                total = self.n - nextCount[small]
                nextCount[medium] = total//2
                nextCount[large] = self.n - (nextCount[medium] + nextCount[small])

        return nextCount

'''
Simulation with a policy that does not depend on state. (Choose the same u everytime.)
'''
class ConstantPolicy(BasePolicy):
    def __init__(self, state, u):
        super().__init__(state)
        self.constantPolicy = u
        self.policyName = "Constant  "

    def generatePlan(self):
        self.plan = [self.constantPolicy[loc.index] - loc.count for loc in self.locs]

def viewPolicies(initState):
    policies = [
        BasePolicy(initState),
        HalfResetPolicy(initState),
        ResetPolicy(initState),
        TwoLimitPolicy(initState),
        ConstantPolicy(initState, [55, 55, 70])]

    for p in policies:
        for _ in range(20):
            p.step()
        p.showState()

'''
Simulation with a pre-calculated policy.
'''
class LookupPolicy(BasePolicy):
    def __init__(self, state, policyDict):
        super().__init__(state)
        self.policy = policyDict
        self.policyName = "Lookup     "

    def generatePlan(self):
        x = [loc.count for loc in self.locs]
        small = x.index(min(x))
        temp = [x[i] if i!= small else -1 for i in range(3)]
        large = temp.index(max(temp))
        medium = 3 - (small + large)
        sortedAction = self.policy[(x[small], x[medium], x[large])]
        u = [0, 0, 0]
        u[small] = sortedAction[0]
        u[medium] = sortedAction[1]
        u[large] = sortedAction[2]
        self.plan = [i - j for i, j in zip(u, x)]


'''
Simulation with a pre-calculated policy.
'''
class mStepLookaheadPolicy(BasePolicy):
    def __init__(self, state, k, m, approximateValuesToGo = {}, twoLimits = (18, 22), trials = 10):
        super().__init__(state)
        self.k = k
        self.m = m
        self.approxValues = approximateValuesToGo
        self.twoLimits = twoLimits
        self.trials = trials
        self.policyName = str(m) + " Step Lookahead"

    def generatePlan(self):
        x = [loc.count for loc in self.locs]
        small = x.index(min(x))
        temp = [x[i] if i!= small else -1 for i in range(3)]
        large = temp.index(max(temp))
        medium = 3 - (small + large)

        sortedState = tuple([x[small], x[medium], x[large]])

        gen = GenericLookahead((17, 24), self.trials)
        gen.J(sortedState, self.k, self.m)


        sortedAction = gen.policy[(sortedState, self.k, self.m)]

        u = [0, 0, 0]
        u[small] = sortedAction[0]
        u[medium] = sortedAction[1]
        u[large] = sortedAction[2]
        self.plan = [i - j for i, j in zip(u, x)]
        self.k -= 1
        self.m = min(self.k, self.m)


def viewPolicies(initState):
    policies = [
        BasePolicy(initState),
        HalfResetPolicy(initState),
        ResetPolicy(initState),
        TwoLimitPolicy(initState),
    ]
    for p in policies:
        for _ in range(20):
            p.step()
        p.showState()


In [219]:
class GenericLookahead:
    def __init__(self, twoLimits, trials = 10):
        self.policy = {} # Chosen u of each (x, k, m)
        self.valuesToGo = {} # J(x, k, m)
        self.expectations = {} # H(u, k, m)
        self.numTrials = trials
        self.twoLimits = twoLimits
        self.approximateValuesToGo = approxValueTable # J_approx(x, k)

    def __str__(self, minStage = 0):
        summary = ""
        for (x, k, m) in self.valuesToGo.keys():
            if k >= minStage:
                if (x, k, m) in self.policy.keys():
                    summary += f"k: {k} m: {m} x: {x} u(x): {self.policy[(x, k, m)]} J(x): {round(self.valuesToGo[x, k, m])} \n"
                else:
                    summary += f"k: {k} m: {m} x: {x} J(x): {round(self.valuesToGo[x, k, m])} \n"
        return summary


    '''
        J(x, k, m) = max_u { H(u, k, m) - transferCost(x-->u) }
        - Find all reasonable u's
        - Get H(u, k) (expected value-to-go starting from u)
        - Subtract cost of transfer from x to u
        - Select u with the largest value-to-go
    '''
    def J(self, x, k, m):
        if (x, k, m) in self.valuesToGo.keys():
             return self.valuesToGo[(x, k, m)]
        if m==0:
          return self.approximateValuesToGo[(x, k)]

        (a, b, c) = x
        n = sum(x)
        center = n//3

        actionValues = {}
        for u in getActionCandidates(x):
            actionValues[u] = self.H(tuple(sorted(u)), k, m) - transferCost(x, u)

        bestAction = max(actionValues, key=actionValues.get)
        self.policy[(x, k, m)] = bestAction
        self.valuesToGo[(x, k, m)] = actionValues[bestAction]

        return (actionValues[bestAction])

    '''
    H(u, k, m) = E[stageValue(u, w) + J(f(u, w), k-1, m-1)]

    - Build many simulations(using the "do nothing" base policy with starting state u)
    - Move all simulations one step forward
    - Get the stage profit of each simulation
    - Get J(nextState, k-1)
    - Return expected value of stage profit + J(nextState, k-1)
    '''

    def H(self, u, k, m):
        if (u, k, m) in self.expectations.keys():
            return self.expectations[(u, k, m)]

        profitSum = 0
        trials = [BasePolicy(u) for _ in range(self.numTrials)]
        for t in trials:
            t.step()
            nextState = t.state()
            stageValue = t.profit
            valueToGo = self.J(tuple(sorted(nextState)), k-1, m-1)
            profitSum += stageValue + valueToGo

        self.expectations[(u, k, m)] = profitSum/self.numTrials
        return self.expectations[(u, k, m)]

    def findFullPolicy(self, k, m):
      for x in stateSpace:
        print(x)
        self.J(x, k, m)

In [220]:
# Helper functions

def printDictionary(d):
  for item in d.items():
    print(item)


def transferCost(x, u):
  COST_PER_CAR = 50
  carsTransfered = 0
  for i in range(3):
      carsTransfered += abs(x[i] - u[i])

  return carsTransfered * COST_PER_CAR / 2


def getActionCandidates(x):
  (a, b, c) = x
  n = sum(x)
  center = n//3
  actionCandidates = [(u_a, u_b, u_c)
          for u_a in range(min(a, center), max(a, center) + 1)
          for u_b in range(min(b, center), max(b, center) + 1)
          for u_c in range(min(c, center), max(c, center) + 1)]
  actionCandidates = list(filter(lambda state: sum(state) == n, actionCandidates))
  return actionCandidates


The state space for this problem (with fleet size n = 60) is every possible distribution of cars: (0, 0, 60), (0, 1, 59), (0, 2, 58), ..., (20, 20, 20). (Since the system is symmetric, we do not distinguish between the ≤ 6 permutations of each state.)

In [221]:
initState = (20, 20, 20)
n = 60
stateSpace = [(a, b, c)
        for a in range(0, n//3 + 1)
        for b in range(a, n+1)
        for c in range(b, n+1)]
stateSpace = list(filter(lambda state: sum(state) == n, stateSpace))

In [222]:
def exploreTwoLimitPolicies(currState, numTrials, numSteps, reach = 10):
    profits = {}
    [a, b, c] = currState
    lowerLimits = list(range(a - reach, a + 1))
    upperLimits = list(range(a, a + reach + 1))

    for lower in lowerLimits:
        for upper in upperLimits:
            profits[(lower, upper)] = twoLimitValueToGo(currState, lower, upper, numTrials, numSteps)

    bestRange = max(profits, key=profits.get)
    return (bestRange, profits[bestRange])


def twoLimitValueToGo(state, lower, upper, numTrials, numSteps, table = {}):
    profitSum = 0
    trials = [TwoLimitPolicy(state, lower, upper) for _ in range(numTrials)]

    for i in range(numSteps):
      for t in trials:
        t.step()
        profitSum += t.profit

    return profitSum/numTrials

We find the best two-limit policy to use for this system. Some initial testing showed that the choice of two-limit policy was not strongly dependent on the time horizon; the best two-limit policy over a month was about the same as the best two-limit policy over a year. So, we use the same two-limit range, [17, 24], for all experiments regardless of time horizon.

In [225]:
limits = exploreTwoLimitPolicies(initState, 100, 360, 6)
print(360, limits)

360 ((17, 24), 503216733.5)


approxValueTable[(x, k)] is the value-to-go using the chosen two-limit approximation policy starting from state x and running for k stages. Each entry is based on the expected value of [(f(x), k+1)], using results already in the table.

In [226]:
approxValueTable = {}
def recursiveTwoLimitApproximation(x, lower, upper, k, numTrials = 100):
  if (x, k) not in approxValueTable.keys():
    if k == 0:
      approxValueTable[(x, k)] = 0
    else:
      trials = [TwoLimitPolicy(x, lower, upper) for _ in range(numTrials)]
      profitSum = 0
      for t in trials:
        t.step()
        nextState = tuple(sorted([loc.count for loc in t.locs]))
        trialProfit = t.profit + recursiveTwoLimitApproximation(nextState, lower, upper, k-1, numTrials)
        profitSum += trialProfit

      approxValueTable[(x, k)] = profitSum/numTrials
  return approxValueTable[(x, k)]

lower = 17
upper = 24

for x in stateSpace:
  value = recursiveTwoLimitApproximation(tuple(x), lower, upper, 365, 100)
  print(x, value//1000)

for i in range(365):
  for x in stateSpace:
    recursiveTwoLimitApproximation(tuple(x), lower, upper, i, 100)

(0, 0, 60) 2825.0
(0, 1, 59) 2825.0
(0, 2, 58) 2825.0
(0, 3, 57) 2825.0
(0, 4, 56) 2825.0
(0, 5, 55) 2825.0
(0, 6, 54) 2825.0
(0, 7, 53) 2825.0
(0, 8, 52) 2825.0
(0, 9, 51) 2825.0
(0, 10, 50) 2825.0
(0, 11, 49) 2825.0
(0, 12, 48) 2825.0
(0, 13, 47) 2825.0
(0, 14, 46) 2825.0
(0, 15, 45) 2826.0
(0, 16, 44) 2826.0
(0, 17, 43) 2826.0
(0, 18, 42) 2826.0
(0, 19, 41) 2826.0
(0, 20, 40) 2826.0
(0, 21, 39) 2826.0
(0, 22, 38) 2826.0
(0, 23, 37) 2826.0
(0, 24, 36) 2826.0
(0, 25, 35) 2826.0
(0, 26, 34) 2826.0
(0, 27, 33) 2826.0
(0, 28, 32) 2826.0
(0, 29, 31) 2826.0
(0, 30, 30) 2826.0
(1, 1, 58) 2825.0
(1, 2, 57) 2825.0
(1, 3, 56) 2825.0
(1, 4, 55) 2825.0
(1, 5, 54) 2825.0
(1, 6, 53) 2825.0
(1, 7, 52) 2825.0
(1, 8, 51) 2825.0
(1, 9, 50) 2825.0
(1, 10, 49) 2825.0
(1, 11, 48) 2826.0
(1, 12, 47) 2826.0
(1, 13, 46) 2826.0
(1, 14, 45) 2826.0
(1, 15, 44) 2826.0
(1, 16, 43) 2826.0
(1, 17, 42) 2826.0
(1, 18, 41) 2826.0
(1, 19, 40) 2826.0
(1, 20, 39) 2826.0
(1, 21, 38) 2826.0
(1, 22, 37) 2826.0
(1, 23, 36) 

In [227]:
# Comparison of policies over a month.

state = (20, 20, 20)
k = 30

exact = mStepLookaheadPolicy(state, k, k, trials=10)
tenStep = mStepLookaheadPolicy(state, k, 10, trials = 50)
fiveStep = mStepLookaheadPolicy(state, k, 5, trials = 50)

policies = [exact, tenStep, fiveStep]
for p in policies:
  for _ in range(k):
    p.step()
  p.showState()
  print(p.profit)
print("Two limit profit ", approxValueTable[(state, k)])

# Comparison of policies over a year.

state = (20, 20, 20)
k = 365

tenStep = mStepLookaheadPolicy(state, k, 10, trials = 50)
fiveStep = mStepLookaheadPolicy(state, k, 5, trials = 50)

policies = [tenStep, fiveStep]
for p in policies:
  for _ in range(k):
    p.step()
  p.showState()
  p.showHistory()
  print(p.profit)
print("Two limit profit ", approxValueTable[(state, k)])

30 Step Lookahead   A (36), B (12), C (12), Profit: $227 K 

227350.0
10 Step Lookahead   A (27), B (23), C (10), Profit: $233 K 

232750.0
5 Step Lookahead   A (26), B (8), C (26), Profit: $227 K 

226800.0
Two limit profit  232751.25015050307
10 Step Lookahead   A (31), B (11), C (18), Profit: $2802 K 

{'x': (20, 20, 20), 'u': (20, 20, 20)}
{'x': (24, 12, 24), 'u': (23, 16, 21)}
{'x': (26, 13, 21), 'u': (21, 19, 20)}
{'x': (13, 39, 8), 'u': (17, 26, 17)}
{'x': (24, 21, 15), 'u': (21, 21, 18)}
{'x': (18, 29, 13), 'u': (19, 25, 16)}
{'x': (16, 25, 19), 'u': (16, 24, 20)}
{'x': (11, 28, 21), 'u': (15, 24, 21)}
{'x': (17, 26, 17), 'u': (18, 23, 19)}
{'x': (22, 22, 16), 'u': (22, 21, 17)}
{'x': (24, 22, 14), 'u': (22, 22, 16)}
{'x': (18, 22, 20), 'u': (18, 22, 20)}
{'x': (9, 31, 20), 'u': (17, 23, 20)}
{'x': (16, 22, 22), 'u': (16, 22, 22)}
{'x': (39, 13, 8), 'u': (25, 18, 17)}
{'x': (26, 14, 20), 'u': (22, 18, 20)}
{'x': (18, 26, 16), 'u': (19, 24, 17)}
{'x': (16, 23, 21), 'u': (17, 22,